In [1]:
!pwd

/gemini/code


In [ ]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple pynvml

In [ ]:
import os
#os.chdir("code")

In [ ]:
#!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple transformers
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple pandas

In [2]:
from modeling_big_bird import customBigBirdModel,customBigBirdForTokenClassification
import json
from BigBertconfig import BigBirdConfig,dict2obj
from transformers import BigBirdModel,BertTokenizerFast,BertTokenizer
import torch
import numpy as np
import pandas as pd
import torch.utils.data as Data
from utilfunction import padandtruncate
import torch.distributed as dist
import torch.optim as optim
import torch.multiprocessing as mp
from torch.multiprocessing import Process
from torch.nn.parallel import DistributedDataParallel as DDP
from sklearn.utils import shuffle
import pynvml
from importlib import reload


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv("/gemini/data-1/trainfilev6.csv")
for i in df.index:
    df.at[i,"xid4"]=eval(df.at[i,"xid4"])
    df.at[i,"yid4"]=eval(df.at[i,"yid4"])
    df.at[i,"tokenidlist"]=eval(df.at[i,"tokenidlist"])
    df.at[i,"positionid3"]=eval(df.at[i,"positionid3"])
    df.at[i,"labellist"]=eval(df.at[i,"labellist"])
df=shuffle(df,random_state=0)
#df=df.iloc[:2700]

In [ ]:
print(df["tokenidlist"][0])

In [ ]:
#check data
m=3001
print(len(df.at[m,"tokenidlist"])==len(df.at[m,"positionid3"]))
print(len(df.at[m,"tokenidlist"])==len(df.at[m,"xid4"]))
print(len(df.at[m,"tokenidlist"])==len(df.at[m,"yid4"]))
print(len(df.at[m,"tokenidlist"])==len(df.at[m,"labellist"]))
print(df.at[m,"labellist"])
#print(df.at[m,"tokenlist"])
labelpos=np.argwhere(np.array(df.at[m,"labellist"])>0)
print(np.take(df.at[m,"tokenlist"],labelpos),end=",")

In [6]:
maxtokenlength=2500#length blocksizepair(2400,40)(2400,60)(2200,40)(2200,55)(2048,32)
xmaxlength=1000
ymaxlength=3000
maxpositionid=2500
#df=shuffle(df,random_state=0)
xidbatch=list(df["xid4"])
yidbatch=list(df["yid4"])
tokenbatch=list(df["tokenidlist"])
xidbatch=[[int(j) if j <xmaxlength else xmaxlength-1 for j in i]for i in xidbatch]
yidbatch=[[int(j) if j <ymaxlength else ymaxlength-1 for j in i]for i in yidbatch]
positionbatch=list(df["positionid3"])
positionbatch=[[int(j) if j <maxpositionid else maxpositionid-1  for j in i]for i in positionbatch]
labelbatch=list(df["labellist"])
lengthlist=[len(i) for i in tokenbatch]

In [7]:
xidbatch=padandtruncate(xidbatch,maxtokenlength,dtype=torch.int32)
yidbatch=padandtruncate(yidbatch,maxtokenlength,dtype=torch.int32)
positionbatch=padandtruncate(positionbatch,maxtokenlength,dtype=torch.int32)
labelbatch=padandtruncate(labelbatch,maxtokenlength,dtype=torch.long,padvalue=0)#padvalue=-100 if not use crf else 0
tokenbatch=padandtruncate(tokenbatch,maxtokenlength,dtype=torch.int32)
maskbatch=tokenbatch>0
maskbatch=maskbatch.type(torch.ByteTensor)
classweight=torch.tensor([1,10,10,10,10,10,10,10,10],dtype=torch.float32)

In [21]:
print(maskbatch[0])

tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8)


In [8]:
torch_dataset = Data.TensorDataset(tokenbatch,positionbatch,xidbatch,yidbatch,labelbatch,maskbatch)
train_set,val_set = torch.utils.data.random_split(torch_dataset, [2800, 512])

In [6]:
with open('config.json', encoding='utf-8') as a:
    # 读取文件
    config = json.load(a)
config = dict2obj(config)

In [16]:
transitionmatrix=torch.tensor([[0,0,-100,0,-100,0,-100,0,-100],
[-100,-100,0,-100,-100,-100,-100,-100,-100],
[0,0,0,0,-100,0,-100,0,-100],
[-100,-100,-100,-100,0,-100,-100,-100,-100],
[0,0,-100,0,0,0,-100,0,-100],
[-100,-100,-100,-100,-100,-100,0,-100,-100],
[0,0,-100,0,-100,0,0,0,-100],
[-100,-100,-100,-100,-100,-100,-100,-100,0],
[0,0,-100,0,-100,0,-100,0,0]])
starttrasition=torch.tensor([0,0,-100,0,-100,0,-100,0,-100])
endtransition=torch.tensor([0,-100,0,-100,0,-100,0,-100,0])

In [17]:
model = customBigBirdForTokenClassification(config)
model.load_state_dict(torch.load("modelsecondtrain.bin"),strict=False)

_IncompatibleKeys(missing_keys=['crf.start_transitions', 'crf.end_transitions', 'crf.transitions'], unexpected_keys=[])

In [19]:
cutomdict=model.state_dict()
cutomdict["crf.transitions"].copy_(transitionmatrix.clone())
cutomdict["crf.start_transitions"].copy_(starttrasition.clone())
cutomdict["crf.end_transitions"].copy_(endtransition.clone())

tensor([   0., -100.,    0., -100.,    0., -100.,    0., -100.,    0.])

In [ ]:
#train process
rank=0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_loader_train = torch.utils.data.DataLoader(dataset=train_set,batch_size=3)

model = customBigBirdForTokenClassification(config)
model.load_state_dict(torch.load("modelthirdtrain.bin"),strict=False)
model= model.to(device)
classweight=classweight.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00004)
accumulation_steps=12
model.train()
j=0
for epoch in range(4):
    print("epoch: "+str(epoch))
    for i,(tokenid,positionid,xid,yid,labelid,masklist) in enumerate(data_loader_train):
        tokenid=tokenid.to(device)
        positionid=positionid.to(device)
        xid=xid.to(device)
        yid=yid.to(device)
        labelid=labelid.to(device)
        masklist=masklist.to(device)
        output = model(input_ids=tokenid,position_ids=positionid,x2d_ids=xid,y2d_ids=yid,labels=labelid,attention_mask=masklist,classweight=classweight)
        #loss = output[0]/accumulation_steps #not use crf
        loss = output/accumulation_steps     #use crf
        loss.backward()
        if((j+1)%accumulation_steps)==0:
            # optimizer the net
            optimizer.step()        # update parameters of net
            optimizer.zero_grad()   # reset gradient
        if i % 50 == 0:
            print("loss: {}".format(loss))
        j=j+1

torch.save(model.state_dict(), "modelfourthtrain.bin")

In [25]:
torch.save(model.state_dict(), "modelsecondtrainwithmanualcrf.bin")

In [ ]:
torch.save(model.state_dict(), "modelv1.bin")

In [9]:
torch.cuda.empty_cache()
#print(torch.cuda.memory_summary())

In [3]:
tokenizer=BertTokenizer.from_pretrained("pretrainconfig")

In [7]:
print(tokenizer.tokenize('报告题目：A Personalized Federated Learning Framework for Heterogeneous Population'))

['报', '告', '题', '目', '：', 'a', 'personal', '##ized', 'fed', '##erated', 'learning', 'framework', 'for', 'het', '##ero', '##gen', '##eous', 'population']


In [18]:
#predict not use crf
model = customBigBirdForTokenClassification(config)
model.load_state_dict(torch.load("modelsecondtrain.bin"))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model= model.to(device)
classweight=classweight.to(device)
data_loader_val = torch.utils.data.DataLoader(dataset=val_set,batch_size=4)
logitlist=[]
tokenidlist=[]
labelidlist=[]
with torch.no_grad():
    model.eval()
    for i,(tokenid,positionid,xid,yid,labelid,masklist) in enumerate(data_loader_val):
        tokenid=tokenid.to(device)
        positionid=positionid.to(device)
        xid=xid.to(device)
        yid=yid.to(device)
        labelid=labelid.to(device)
        masklist=masklist.to(device)
        output = model(input_ids=tokenid,position_ids=positionid,x2d_ids=xid,y2d_ids=yid,labels=labelid,attention_mask=masklist,classweight=classweight)
        logitlist.append(torch.argmax(output[1],dim=2))
        tokenidlist.append(tokenid)
        labelidlist.append(labelid)
logitlist=torch.cat(logitlist,dim=0).cpu().detach().numpy()
tokenidlist=torch.cat(tokenidlist,dim=0)
labelidlist=torch.cat(labelidlist,dim=0).cpu().detach().numpy()

In [21]:
dfpredict=pd.DataFrame(columns=["token/predict/true"])
for i in range(tokenidlist.shape[0]):
    tokenlist=tokenizer.convert_ids_to_tokens(tokenidlist[i])
    string=""
    for j in range(len(tokenidlist[i])):
        string=string+tokenlist[j]+"/"+str(logitlist[i][j])+"/"+str(labelidlist[i][j])+" "
    dfpredict.loc[i]=[string]
dfpredict.to_csv("predictlist.csv")

In [2]:
df=pd.read_csv("/gemini/data-3/predictfile.csv")
for i in df.index:
    df.at[i,"xid"]=eval(df.at[i,"xid"])
    df.at[i,"yid"]=eval(df.at[i,"yid"])
    df.at[i,"tokenid"]=eval(df.at[i,"tokenid"])
    df.at[i,"positionid"]=eval(df.at[i,"positionid"])
df=shuffle(df,random_state=0)

In [3]:
maxtokenlength=2500#length blocksizepair(2400,40)(2400,60)(2200,40)(2200,55)(2048,32)
xmaxlength=1000
ymaxlength=3000
maxpositionid=2500
#df=shuffle(df,random_state=0)
xidbatch=list(df["xid"])
yidbatch=list(df["yid"])
tokenbatch=list(df["tokenid"])
xidbatch=[[int(j) if j <xmaxlength else xmaxlength-1 for j in i]for i in xidbatch]
yidbatch=[[int(j) if j <ymaxlength else ymaxlength-1 for j in i]for i in yidbatch]
positionbatch=list(df["positionid"])
positionbatch=[[int(j) if j <maxpositionid else maxpositionid-1  for j in i]for i in positionbatch]
lengthlist=[len(i) for i in tokenbatch]

xidbatch=padandtruncate(xidbatch,maxtokenlength,dtype=torch.int32)
yidbatch=padandtruncate(yidbatch,maxtokenlength,dtype=torch.int32)
positionbatch=padandtruncate(positionbatch,maxtokenlength,dtype=torch.int32)
tokenbatch=padandtruncate(tokenbatch,maxtokenlength,dtype=torch.int32)
maskbatch=tokenbatch>0
maskbatch=maskbatch.type(torch.ByteTensor)
classweight=torch.tensor([1,10,10,10,10,10,10,10,10],dtype=torch.float32)

In [4]:
predict_dataset = Data.TensorDataset(tokenbatch,positionbatch,xidbatch,yidbatch,maskbatch)


In [8]:
#predict use crf
model = customBigBirdForTokenClassification(config)
model.load_state_dict(torch.load("modelsecondtrainwithmanualcrf.bin"))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model= model.to(device)
classweight=classweight.to(device)
data_loader_val = torch.utils.data.DataLoader(dataset=predict_dataset,batch_size=4)
#logitlist=[]
tokenidlist=[]
logitlist=[]
#labelidlist=[]
with torch.no_grad():
    model.eval()
    for i,(tokenid,positionid,xid,yid,masklist) in enumerate(data_loader_val):
        tokenid=tokenid.to(device)
        positionid=positionid.to(device)
        xid=xid.to(device)
        yid=yid.to(device)
        #labelid=labelid.to(device)
        masklist=masklist.to(device)
        output = model(input_ids=tokenid,position_ids=positionid,x2d_ids=xid,y2d_ids=yid,attention_mask=masklist,classweight=classweight)#labels=labelid
        tokenidlist.append(tokenid)
        logitlist.append(output)
tokenidlist=torch.cat(tokenidlist,dim=0)
logitlist=[j for i in range(len(logitlist)) for j in logitlist[i]]
#labelidlist=torch.cat(labelidlist,dim=0).cpu().detach().numpy()

In [9]:
print(len(logitlist[0]))
print(len(tokenidlist[0]))

771
2500


In [10]:
dfpredict=pd.DataFrame(columns=["tokenwithid"])
for i in range(tokenidlist.shape[0]):
    tokenlist=tokenizer.convert_ids_to_tokens(tokenidlist[i])
    string=""
    for j in range(len(logitlist[i])):
        string=string+tokenlist[j]+"/"+str(logitlist[i][j])+" "
    dfpredict.loc[i]=[string,]
dfpredict.to_csv("predictnewfilelistwithcrf.csv")

In [28]:
logitlist=logitlist.cpu().detach().numpy()

In [30]:
dfpredict.to_csv("predictlist.csv",encoding='utf_8_sig')

In [36]:
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0) # 0表示显卡标号
meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
print(meminfo.total/1024**2) #总的显存大小
print(meminfo.used/1024**2)  #已用显存大小
print(meminfo.free/1024**2)  #剩余显存大小

24258.0
4247.265625
20010.734375
